<a href="https://colab.research.google.com/github/kiril-buga/Neural-Network-Training-Project/blob/main/1D_CNN_Multilabel_V2/Y_1d_CNN_5_Labels_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## 📚 Quick Start Guide

### First Time Setup:
1. **Run all cells in order** from top to bottom
2. **Configure settings** in the Configuration section (Cell 3-4)
   - Set `TOTAL_EPOCHS`, `BATCH_SIZE`, etc.
   - Enable `UPLOAD_TO_HF` if you want automatic checkpoint uploads
3. **Optional**: Run the HuggingFace login cell if uploading checkpoints

### To Resume Training:
1. Set `RESUME_FROM_CHECKPOINT = True` in configuration
2. Set `CHECKPOINT_TO_LOAD` to your checkpoint file (e.g., "checkpoint_epoch_025.pkl")
3. If checkpoint is on HuggingFace, set `DOWNLOAD_FROM_HF = True`
4. Run all cells from configuration onwards

### To Change Model Architecture:
Edit these variables in the Configuration section:
- `CONV_FILTERS = [32, 64, 128, 256]` - Number of filters per conv block
- `KERNEL_SIZE = 3` - Convolution kernel size
- `DROPOUT_CONV = 0.2` - Dropout rate after convolutions
- `DROPOUT_DENSE = 0.3` - Dropout rate in dense layers
- `DENSE_LAYERS = [256, 128]` - Dense layer sizes

### Key Features:
- ✅ **Centralized Configuration** - All settings in one place
- ✅ **Automatic Checkpointing** - Saves every N epochs
- ✅ **HuggingFace Integration** - Optional cloud backup
- ✅ **Resumable Training** - Continue from any checkpoint
- ✅ **Threshold Optimization** - Per-class threshold tuning
- ✅ **Comprehensive Metrics** - F1, ROC-AUC, confusion matrices

### Troubleshooting:
- **Out of memory**: Reduce `BATCH_SIZE`
- **Training too slow**: Increase `BATCH_SIZE` (if memory allows)
- **Overfitting**: Increase `DROPOUT_CONV` and `DROPOUT_DENSE`
- **Underfitting**: Decrease dropout or train more epochs

# ECG Multi-Label Model Training (Self-Contained)
Fully independent notebook: loads preprocessed data, trains multi-label model, saves results

### Code Block 1: Package Installation


In [47]:
!pip install tensorflow scikit-learn matplotlib seaborn huggingface-hub h5py -q

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    hamming_loss, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, auc, confusion_matrix,
    accuracy_score, jaccard_score
)


# Note: Random seed and directory setup happens in the next cell


### Code Block 2: Configuration Settings


In [48]:
# ============================================================================
# 🔧 MASTER CONFIGURATION - Edit these values to customize training
# ============================================================================

# === PATHS ===
DATA_DIR_NAME = "multilabel_v2"
RESULTS_DIR = "./results/"
CHECKPOINT_DIR = "./checkpoints/"

# === HUGGING FACE ===
HF_USERNAME = "kiril-buga"
HF_DATASET_REPO = "kiril-buga/ECG-database"
HF_MODEL_REPO = "Neural-Network-Project/ECG-models"  # Single source of truth
UPLOAD_TO_HF = True  # Set True to auto-upload checkpoints during training
DOWNLOAD_FROM_HF = False  # Set True to download checkpoint from HuggingFace

# === TRAINING PARAMETERS ===
BATCH_SIZE = 64
TOTAL_EPOCHS = 50
LEARNING_RATE = 1e-3
RANDOM_SEED = 42

# === CHECKPOINT SETTINGS ===
RESUME_FROM_CHECKPOINT = True  # Set True to resume from a checkpoint
CHECKPOINT_TO_LOAD = "checkpoint_epoch_021.pkl"  # Which checkpoint to load
CHECKPOINT_FREQ = 3  # Save checkpoint every N epochs
CHECKPOINT_KEEP_LAST = 3  # Keep only last N checkpoints to save space

# === DATA SPLITS ===
TEST_SIZE = 0.2  # 20% of data for test set
VAL_SIZE = 0.25  # 25% of remaining data for validation

# === MODEL ARCHITECTURE ===
CONV_FILTERS = [32, 64, 128, 256]  # Number of filters per conv block
KERNEL_SIZE = 3  # Kernel size for all conv layers
DROPOUT_CONV = 0.2  # Dropout rate after conv blocks
DROPOUT_DENSE = 0.3  # Dropout rate in dense layers
DENSE_LAYERS = [256, 128]  # Dense layer sizes before output

# === TRAINING CALLBACKS ===
EARLY_STOPPING_PATIENCE = 5  # Stop if no improvement for N epochs
REDUCE_LR_FACTOR = 0.5  # Multiply LR by this factor when reducing
REDUCE_LR_PATIENCE = 5  # Reduce LR if no improvement for N epochs
MIN_LEARNING_RATE = 1e-6  # Minimum learning rate

# === THRESHOLD OPTIMIZATION ===
DEFAULT_THRESHOLD = 0.5  # Default classification threshold
MIN_RECALL_TARGETS = {  # Minimum recall targets per disease
    'Myocarditis': 0.80,
    'Cardiomyopathy': 0.85,
    'Kawasaki': 0.85,
    'CHD': 0.80,
    'Healthy': 0.95
}
THRESHOLD_SEARCH_START = 0.1  # Start of threshold search range
THRESHOLD_SEARCH_END = 0.91  # End of threshold search range
THRESHOLD_SEARCH_STEP = 0.01  # Step size for threshold search

# === VISUALIZATION ===
DPI = 150  # DPI for saved figures
FIGSIZE_LARGE = (16, 12)  # Figure size for large plots
FIGSIZE_MEDIUM = (14, 5)  # Figure size for medium plots
FIGSIZE_ROC = (15, 10)  # Figure size for ROC curves

# ============================================================================
# Print configuration summary
# ============================================================================
print("="*80)
print("CONFIGURATION SUMMARY")
print("="*80)
print(f"\n📁 Paths:")
print(f"  Data directory: {DATA_DIR_NAME}")
print(f"  Results: {RESULTS_DIR}")
print(f"  Checkpoints: {CHECKPOINT_DIR}")
print(f"\n🤗 Hugging Face:")
print(f"  Dataset repo: {HF_DATASET_REPO}")
print(f"  Model repo: {HF_MODEL_REPO}")
if UPLOAD_TO_HF:
    print(f"  ⚠️  Make sure repo exists: https://huggingface.co/{HF_MODEL_REPO}")
print(f"  Auto-upload: {UPLOAD_TO_HF}")
print(f"\n🎯 Training:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Total epochs: {TOTAL_EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Random seed: {RANDOM_SEED}")
print(f"\n💾 Checkpoints:")
print(f"  Resume training: {RESUME_FROM_CHECKPOINT}")
if RESUME_FROM_CHECKPOINT:
    print(f"  Load from: {CHECKPOINT_TO_LOAD}")
    print(f"  Download from HF: {DOWNLOAD_FROM_HF}")
print(f"  Save frequency: every {CHECKPOINT_FREQ} epochs")
print(f"  Keep last: {CHECKPOINT_KEEP_LAST} checkpoints")
print(f"\n🏗️ Model Architecture:")
print(f"  Conv filters: {CONV_FILTERS}")
print(f"  Kernel size: {KERNEL_SIZE}")
print(f"  Dropout (conv): {DROPOUT_CONV}")
print(f"  Dropout (dense): {DROPOUT_DENSE}")
print(f"  Dense layers: {DENSE_LAYERS}")
print("\n" + "="*80)

CONFIGURATION SUMMARY

📁 Paths:
  Data directory: multilabel_v2
  Results: ./results/
  Checkpoints: ./checkpoints/

🤗 Hugging Face:
  Dataset repo: kiril-buga/ECG-database
  Model repo: Neural-Network-Project/ECG-models
  ⚠️  Make sure repo exists: https://huggingface.co/Neural-Network-Project/ECG-models
  Auto-upload: True

🎯 Training:
  Batch size: 64
  Total epochs: 50
  Learning rate: 0.001
  Random seed: 42

💾 Checkpoints:
  Resume training: True
  Load from: checkpoint_epoch_021.pkl
  Download from HF: False
  Save frequency: every 3 epochs
  Keep last: 3 checkpoints

🏗️ Model Architecture:
  Conv filters: [32, 64, 128, 256]
  Kernel size: 3
  Dropout (conv): 0.2
  Dropout (dense): 0.3
  Dense layers: [256, 128]



## 🔧 Configuration

All configurable parameters are centralized here. Edit these values to customize your training run.

## HuggingFace Authentication

**Run this cell once** if you plan to upload checkpoints to HuggingFace or download from private repos.


### Code Block 3: Configuration Settings


In [49]:
# Apply random seed configuration for reproducibility
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Create directories
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print(f"✓ Random seed set to: {RANDOM_SEED}")
print(f"✓ TensorFlow version: {tf.__version__}")
print(f"✓ GPU devices: {tf.config.list_physical_devices('GPU')}")
print(f"✓ Directories created: {RESULTS_DIR}, {CHECKPOINT_DIR}")

✓ Random seed set to: 42
✓ TensorFlow version: 2.19.0
✓ GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✓ Directories created: ./results/, ./checkpoints/


### Code Block 4: Import Dependencies


In [50]:
# Login to HuggingFace (matches preprocessing approach)
if UPLOAD_TO_HF:
    from huggingface_hub import login
    print("Logging into Hugging Face...")
    login()
    print("Logged in!")
    print(f"Will upload to: {HF_MODEL_REPO}")
else:
    print("Skipping HF login (UPLOAD_TO_HF = False)")


Logging into Hugging Face...


Logged in!
Will upload to: Neural-Network-Project/ECG-models


## Environment & Data Loading

## Data Loading Functions

Utility functions for loading and preparing the ECG dataset.

### Code Block 5: Data Loading Function


In [51]:
def create_dataset(h5_path, indices, batch_size=32, is_training=True):
    """Create streaming dataset from HDF5.

    NOTE: HDF5 requires indices in increasing order for fancy indexing.
    We sort indices before access and unsort after to maintain shuffling.
    """
    def gen():
        with h5py.File(h5_path, 'r') as h5f:
            X_dset = h5f['X']
            y_dset = h5f['y']

            # Shuffle for training, don't shuffle for val/test
            idx = np.random.permutation(indices) if is_training else indices

            for i in range(0, len(idx), batch_size):
                batch_idx = idx[i:i+batch_size]

                # HDF5 requires sorted indices for fancy indexing
                # So we sort, access, then unsort to maintain shuffle
                batch_idx_sorted = np.sort(batch_idx)
                X_batch = X_dset[batch_idx_sorted]
                y_batch = y_dset[batch_idx_sorted]

                # Create unsort permutation to restore original shuffle order
                unsort_idx = np.argsort(np.argsort(batch_idx))
                X_batch = X_batch[unsort_idx]
                y_batch = y_batch[unsort_idx]

                # Convert float16 to float32 if needed
                if X_batch.dtype == np.float16:
                    X_batch = X_batch.astype(np.float32)

                yield X_batch, y_batch

    dataset = tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(None, *input_shape), dtype=tf.float32),
            tf.TensorSpec(shape=(None, len(DISEASE_CLASSES)), dtype=tf.int32)
        )
    )

    return dataset.prefetch(tf.data.AUTOTUNE)

# Create datasets


### Code Block 6: Import Dependencies


In [52]:
# Download preprocessed data from Hugging Face
from huggingface_hub import snapshot_download
import h5py

print("Downloading preprocessed data from Hugging Face...")
local_dir = snapshot_download(
    repo_id=HF_DATASET_REPO,
    repo_type="dataset",
    local_dir="./ECG-database/",
    allow_patterns=[f"{DATA_DIR_NAME}/*"]  # Only download preprocessed data (~1.5-2 GB)
)

DATA_DIR = os.path.join(local_dir, DATA_DIR_NAME)
# RESULTS_DIR defined in config
# Directories created in config apply cell

# HF_MODEL_REPO is defined in the configuration section
CHECKPOINT_DIR = "./checkpoints/"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print(f"✓ Downloaded to: {DATA_DIR}")
print(f"✓ Results will be saved to: {RESULTS_DIR}")

# ============================================================================
# Load dataset structure from HDF5 and create streaming data generator
h5_path = os.path.join(DATA_DIR, "ecg_data.h5")

print(f"\nOpening HDF5 file: {h5_path}")
with h5py.File(h5_path, 'r') as h5f:
    n_samples = h5f['X'].shape[0]
    input_shape = h5f['X'].shape[1:]
    DISEASE_CLASSES = list(h5f.attrs['disease_classes'])
    data_format = h5f.attrs.get('data_format', 'unknown')

print(f"✓ Dataset info:")
print(f"  Total windows: {n_samples}")
print(f"  Input shape: {input_shape}")
print(f"  Data format: {data_format}")
print(f"  Classes: {DISEASE_CLASSES}")

# Create train/val/test split indices (without loading data)
all_idx = np.arange(n_samples)
train_idx, test_idx = train_test_split(all_idx, test_size=TEST_SIZE, random_state=42)
train_idx, val_idx = train_test_split(train_idx, test_size=TEST_SIZE, random_state=42)

print(f"\nTrain: {len(train_idx)}, Val: {len(val_idx)}, Test: {len(test_idx)}")

# Create tf.data.Dataset objects
print("\nCreating streaming datasets...")
# BATCH_SIZE defined in config
train_dataset = create_dataset(h5_path, train_idx, batch_size=BATCH_SIZE, is_training=True)
val_dataset = create_dataset(h5_path, val_idx, batch_size=BATCH_SIZE, is_training=False)
test_dataset = create_dataset(h5_path, test_idx, batch_size=BATCH_SIZE, is_training=False)

print(f"✓ Datasets created (batch_size={BATCH_SIZE})")

# Verify dataset contents
print("\nVerifying datasets...")
for batch_X, batch_y in train_dataset.take(1):
    print(f"  Sample batch - X shape: {batch_X.shape}, y shape: {batch_y.shape}")
    for i, cls in enumerate(DISEASE_CLASSES):
        count = int(tf.reduce_sum(batch_y[:, i]).numpy())
        print(f"    {cls}: {count} in batch")
    print("✓ Dataset verification successful!")

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

✓ Downloaded to: /content/ECG-database/multilabel_v2
✓ Results will be saved to: ./results/

Opening HDF5 file: /content/ECG-database/multilabel_v2/ecg_data.h5
✓ Dataset info:
  Total windows: 59944
  Input shape: (5000, 12)
  Data format: unknown
  Classes: ['Myocarditis', 'Cardiomyopathy', 'Kawasaki', 'CHD', 'Healthy']

Train: 38364, Val: 9591, Test: 11989

Creating streaming datasets...
✓ Datasets created (batch_size=64)

Verifying datasets...
  Sample batch - X shape: (64, 5000, 12), y shape: (64, 5)
    Myocarditis: 1 in batch
    Cardiomyopathy: 0 in batch
    Kawasaki: 0 in batch
    CHD: 5 in batch
    Healthy: 58 in batch
✓ Dataset verification successful!


## Build & Train Model

### Code Block 7: Model Architecture Definition


In [53]:
# Build multi-label 1D CNN model
def build_model(input_shape, num_classes):
        return keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(CONV_FILTERS[0], KERNEL_SIZE, padding='same', activation='relu'),
        layers.Conv1D(CONV_FILTERS[0], KERNEL_SIZE, padding='same', activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(DROPOUT_CONV),
        layers.Conv1D(CONV_FILTERS[1], KERNEL_SIZE, padding='same', activation='relu'),
        layers.Conv1D(CONV_FILTERS[1], KERNEL_SIZE, padding='same', activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(DROPOUT_CONV),
        layers.Conv1D(CONV_FILTERS[2], KERNEL_SIZE, padding='same', activation='relu'),
        layers.Conv1D(CONV_FILTERS[2], KERNEL_SIZE, padding='same', activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(DROPOUT_CONV),
        layers.Conv1D(CONV_FILTERS[3], KERNEL_SIZE, padding='same', activation='relu'),
        layers.Conv1D(CONV_FILTERS[3], KERNEL_SIZE, padding='same', activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(DROPOUT_CONV),
        layers.GlobalAveragePooling1D(),
        layers.Dense(DENSE_LAYERS[0], activation='relu'),
        layers.Dropout(DROPOUT_DENSE),
        layers.Dense(DENSE_LAYERS[1], activation='relu'),
        layers.Dropout(DROPOUT_DENSE),
        layers.Dense(num_classes, activation='sigmoid')  # Multi-label: sigmoid
    ])

print("Building model...")
model = build_model(input_shape, len(DISEASE_CLASSES))

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',  # Multi-label loss
    metrics=['binary_accuracy']
)

print("✓ Model built")
model.summary()

Building model...
✓ Model built


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_64 (Conv1D)              │ (None, 5000, 32)       │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_65 (Conv1D)              │ (None, 5000, 32)       │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_32 (MaxPooling1D) │ (None, 2500, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_48 (Dropout)            │ (None, 2500, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_66 (Conv1D)              │ (None, 2500, 64)       │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_67 (Conv1D)              │ (None, 2500, 64)       │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_33 (MaxPooling1D) │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_49 (Dropout)            │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_68 (Conv1D)              │ (None, 1250, 128)      │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_69 (Conv1D)              │ (None, 1250, 128)      │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_34 (MaxPooling1D) │ (None, 625, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_50 (Dropout)            │ (None, 625, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_70 (Conv1D)              │ (None, 625, 256)       │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_71 (Conv1D)              │ (None, 625, 256)       │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_35 (MaxPooling1D) │ (None, 312, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_51 (Dropout)            │ (None, 312, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_8      │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_52 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_53 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 491,589 (1.88 MB)

 Trainable params: 491,589 (1.88 MB)

 Non-trainable params: 0 (0.00 B)

### Code Block 8: Initialize Model & Load Checkpoint


In [54]:
# ============================================================================
# BUILD MODEL AND OPTIONALLY LOAD CHECKPOINT
# ============================================================================

# Build model
print("Building model...")
model = build_model(input_shape, len(DISEASE_CLASSES))

# Create optimizer
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

# Compile model
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

print("✓ Model built and compiled")
model.summary()

# Variables to track training state
initial_epoch = 0
loaded_history = None

# Load checkpoint if resuming
if RESUME_FROM_CHECKPOINT:
    print("\n" + "="*80)
    print("LOADING CHECKPOINT TO RESUME TRAINING")
    print("="*80)

    checkpoint_path = os.path.join(CHECKPOINT_DIR, CHECKPOINT_TO_LOAD)

    # Download from Hugging Face if requested
    if DOWNLOAD_FROM_HF:
        try:
            checkpoint_path = download_from_huggingface(
                HF_MODEL_REPO,
                CHECKPOINT_TO_LOAD,
                CHECKPOINT_DIR
            )
        except Exception as e:
            print(f"Failed to download from HF: {e}")
            print("Trying local checkpoint...")

    # Load the checkpoint
    if os.path.exists(checkpoint_path):
        try:
            initial_epoch, loaded_history = load_checkpoint(checkpoint_path, model, optimizer)
            print(f"\n✓ Successfully loaded checkpoint!")
            print(f"  Resuming from epoch: {initial_epoch}")
            print(f"  Will train until epoch: {TOTAL_EPOCHS}")
            print(f"  Remaining epochs: {TOTAL_EPOCHS - initial_epoch}")

            # Print last few training metrics
            if loaded_history:
                print(f"\n  Last training metrics:")
                print(f"    Loss: {loaded_history['loss'][-1]:.4f}")
                print(f"    Val Loss: {loaded_history['val_loss'][-1]:.4f}")
                print(f"    Accuracy: {loaded_history['binary_accuracy'][-1]:.4f}")
                print(f"    Val Accuracy: {loaded_history['val_binary_accuracy'][-1]:.4f}")
        except Exception as e:
            print(f"✗ Failed to load checkpoint: {e}")
            print("Starting training from scratch...")
            initial_epoch = 0
            loaded_history = None
    else:
        print(f"✗ Checkpoint not found: {checkpoint_path}")
        print("Starting training from scratch...")
        initial_epoch = 0
        loaded_history = None

print("\n" + "="*80)

Building model...
✓ Model built and compiled


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_72 (Conv1D)              │ (None, 5000, 32)       │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_73 (Conv1D)              │ (None, 5000, 32)       │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_36 (MaxPooling1D) │ (None, 2500, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_54 (Dropout)            │ (None, 2500, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_74 (Conv1D)              │ (None, 2500, 64)       │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_75 (Conv1D)              │ (None, 2500, 64)       │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_37 (MaxPooling1D) │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_55 (Dropout)            │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_76 (Conv1D)              │ (None, 1250, 128)      │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_77 (Conv1D)              │ (None, 1250, 128)      │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_38 (MaxPooling1D) │ (None, 625, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_56 (Dropout)            │ (None, 625, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_78 (Conv1D)              │ (None, 625, 256)       │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_79 (Conv1D)              │ (None, 625, 256)       │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_39 (MaxPooling1D) │ (None, 312, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_57 (Dropout)            │ (None, 312, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_9      │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_58 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_59 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 491,589 (1.88 MB)

 Trainable params: 491,589 (1.88 MB)

 Non-trainable params: 0 (0.00 B)


LOADING CHECKPOINT TO RESUME TRAINING
Loading checkpoint from ./checkpoints/checkpoint_epoch_021.pkl...
Note: No optimizer weights in checkpoint (will restart optimizer)
✓ Checkpoint loaded: epoch 21

✓ Successfully loaded checkpoint!
  Resuming from epoch: 21
  Will train until epoch: 50
  Remaining epochs: 29

  Last training metrics:
    Loss: 0.0941
    Val Loss: 0.1162
    Accuracy: 0.9674
    Val Accuracy: 0.9606



### Code Block: Checkpoint Management Functions


In [55]:
# Checkpoint utility functions
import pickle
from huggingface_hub import HfApi, hf_hub_download
from pathlib import Path

def save_checkpoint(model, optimizer, epoch, history, checkpoint_path):
    """
    Save a complete training checkpoint including:
    - Model weights
    - Optimizer state (for proper training resumption)
    - Current epoch number
    - Training history
    """
    print(f"Saving checkpoint to {checkpoint_path}...")

    # Check if optimizer has been built (has weights)
    try:
        optimizer_weights = optimizer.get_weights()
        has_optimizer_weights = True
    except (AttributeError, ValueError):
        # Optimizer not built yet or no weights
        optimizer_weights = None
        has_optimizer_weights = False
        print("  Note: Optimizer weights not available yet (first epoch not complete)")

    # Save everything as a dictionary
    checkpoint = {
        'epoch': epoch,
        'model_weights': model.get_weights(),
        'optimizer_config': optimizer.get_config(),
        'optimizer_weights': optimizer_weights,
        'has_optimizer_weights': has_optimizer_weights,
        'history': history
    }

    with open(checkpoint_path, "wb") as f:
        pickle.dump(checkpoint, f)

    # Also save the model separately in Keras format (more portable)
    model_path = checkpoint_path.replace(".pkl", ".keras")
    model.save(model_path)

    print(f"✓ Checkpoint saved: {checkpoint_path}")
    print(f"✓ Model saved: {model_path}")

    return checkpoint_path, model_path


def load_checkpoint(checkpoint_path, model, optimizer):
    """
    Load a complete training checkpoint and restore training state.

    Returns:
        tuple: (epoch, history) - Resume from this epoch with this history
    """
    print(f"Loading checkpoint from {checkpoint_path}...")

    with open(checkpoint_path, "rb") as f:
        checkpoint = pickle.load(f)

    # Restore model weights
    model.set_weights(checkpoint["model_weights"])

    # Restore optimizer state if available
    if checkpoint.get("has_optimizer_weights", False) and checkpoint["optimizer_weights"] is not None:
        # Requires a dummy training step first to initialize optimizer variables
        dummy_batch = tf.zeros((1, *input_shape))
        with tf.GradientTape() as tape:
            _ = model(dummy_batch, training=True)
            dummy_loss = tf.reduce_mean(model.losses) if model.losses else tf.constant(0.0)

        grads = tape.gradient(dummy_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Now we can restore optimizer weights
        try:
            optimizer.set_weights(checkpoint["optimizer_weights"])
            print("✓ Optimizer state restored")
        except Exception as e:
            print(f"Warning: Could not restore optimizer weights: {e}")
    else:
        print("Note: No optimizer weights in checkpoint (will restart optimizer)")

    print(f"✓ Checkpoint loaded: epoch {checkpoint['epoch']}")

    return checkpoint["epoch"], checkpoint["history"]


def upload_to_huggingface(file_path, repo_id, path_in_repo=None, token=None):
    """
    Upload file to HuggingFace Hub (matches working preprocessing approach).
    Repository must exist before uploading.
    """
    from huggingface_hub import HfApi
    from pathlib import Path

    if path_in_repo is None:
        path_in_repo = Path(file_path).name

    print(f"Uploading {file_path} to {repo_id}/{path_in_repo}...")

    api = HfApi()
    api.upload_file(
        path_or_fileobj=file_path,
        path_in_repo=path_in_repo,
        repo_id=repo_id,
        repo_type="model"
    )

    print(f"Uploaded to: https://huggingface.co/{repo_id}")
    print(f"  File: {path_in_repo}")



def download_from_huggingface(repo_id, filename, local_dir="./checkpoints/", token=None):
    """
    Download a file from Hugging Face Hub.

    Args:
        repo_id: Hugging Face repo ID (e.g., "username/model-name")
        filename: Name of file to download
        local_dir: Local directory to save to
        token: HF token (will use HF_TOKEN env variable if not provided)

    Returns:
        str: Path to downloaded file
    """
    print(f"Downloading {filename} from {repo_id}...")

    try:
        file_path = hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            repo_type="model",
            local_dir=local_dir,
            token=token
        )

        print(f"✓ Downloaded to: {file_path}")
        return file_path

    except Exception as e:
        print(f"✗ Download failed: {e}")
        raise


print("✓ Checkpoint management functions loaded")


✓ Checkpoint management functions loaded


In [56]:
# Custom callback for periodic checkpoint saving and HF upload
class CheckpointAndUploadCallback(keras.callbacks.Callback):
    """
    Custom callback that:
    1. Saves complete checkpoints every N epochs
    2. Optionally uploads to Hugging Face Hub
    3. Keeps only the last K checkpoints to save space
    """

    def __init__(self, checkpoint_dir, save_freq=5, upload_to_hf=False,
                 hf_repo_id=None, keep_last=3):
        """
        Args:
            checkpoint_dir: Directory to save checkpoints
            save_freq: Save checkpoint every N epochs
            upload_to_hf: Whether to upload to Hugging Face Hub
            hf_repo_id: Hugging Face repo ID (required if upload_to_hf=True)
            keep_last: Keep only the last N checkpoints (None = keep all)
        """
        super().__init__()
        self.checkpoint_dir = checkpoint_dir
        self.save_freq = save_freq
        self.upload_to_hf = upload_to_hf
        self.hf_repo_id = hf_repo_id
        self.keep_last = keep_last
        self.checkpoints = []  # List of (epoch, checkpoint_path, model_path)

    def on_epoch_end(self, epoch, logs=None):
        # Save checkpoint every save_freq epochs
        if (epoch + 1) % self.save_freq == 0:
            checkpoint_path = os.path.join(
                self.checkpoint_dir,
                f"checkpoint_epoch_{epoch+1:03d}.pkl"
            )

            # Convert history to dictionary format
            history_dict = {k: v for k, v in self.model.history.history.items()}

            # Save checkpoint
            checkpoint_path, model_path = save_checkpoint(
                self.model,
                self.model.optimizer,
                epoch + 1,
                history_dict,
                checkpoint_path
            )

            self.checkpoints.append((epoch + 1, checkpoint_path, model_path))

            # Upload to Hugging Face if enabled
            if self.upload_to_hf and self.hf_repo_id:
                try:
                    print(f"Uploading checkpoint to Hugging Face...")
                    upload_to_huggingface(checkpoint_path, self.hf_repo_id)
                    upload_to_huggingface(model_path, self.hf_repo_id)
                except Exception as e:
                    print(f"Warning: HF upload failed: {e}")

            # Clean up old checkpoints if keep_last is set
            if self.keep_last and len(self.checkpoints) > self.keep_last:
                old_epoch, old_checkpoint, old_model = self.checkpoints.pop(0)
                try:
                    if os.path.exists(old_checkpoint):
                        os.remove(old_checkpoint)
                    if os.path.exists(old_model):
                        os.remove(old_model)
                    print(f"Cleaned up old checkpoint from epoch {old_epoch}")
                except Exception as e:
                    print(f"Warning: Could not remove old checkpoint: {e}")

    def on_train_end(self, logs=None):
        # Save final checkpoint
        checkpoint_path = os.path.join(
            self.checkpoint_dir,
            "checkpoint_final.pkl"
        )

        history_dict = {k: v for k, v in self.model.history.history.items()}

        checkpoint_path, model_path = save_checkpoint(
            self.model,
            self.model.optimizer,
            len(self.model.history.history['loss']),
            history_dict,
            checkpoint_path
        )

        # Upload final checkpoint
        if self.upload_to_hf and self.hf_repo_id:
            try:
                print(f"Uploading final checkpoint to Hugging Face...")
                upload_to_huggingface(checkpoint_path, self.hf_repo_id)
                upload_to_huggingface(model_path, self.hf_repo_id)

                # Also save model card
                self._save_model_card(model_path)
            except Exception as e:
                print(f"Warning: HF upload failed: {e}")

    def _save_model_card(self, model_path):
        """Create a README.md model card for the Hugging Face repo"""
        try:
            model_card = f"""---
tags:
- ecg
- multi-label-classification
- medical
- cardiology
library_name: tensorflow
---

# ECG Multi-Label Classification Model

This model performs multi-label classification on ECG signals to detect:
- Myocarditis
- Cardiomyopathy
- Kawasaki Disease
- Congenital Heart Disease (CHD)
- Healthy

## Model Architecture
- 1D CNN with 4 convolutional blocks
- Input: 12-lead ECG (5000 samples × 12 leads)
- Output: 5 sigmoid outputs (multi-label)

## Training
- Framework: TensorFlow/Keras
- Optimizer: Adam
- Loss: Binary Crossentropy
- Dataset: Pediatric ECG database

## Usage

```python
import tensorflow as tf
from huggingface_hub import hf_hub_download

# Download model
model_path = hf_hub_download(
    repo_id="{self.hf_repo_id}",
    filename="checkpoint_final.keras"
)

# Load model
model = tf.keras.models.load_model(model_path)

# Predict (input shape: [batch_size, 5000, 12])
predictions = model.predict(ecg_data)
```

## Classes
0. Myocarditis
1. Cardiomyopathy
2. Kawasaki Disease
3. CHD
4. Healthy

## Citation
Please cite this model if you use it in your research.
"""

            readme_path = os.path.join(self.checkpoint_dir, "README.md")
            with open(readme_path, 'w') as f:
                f.write(model_card)

            upload_to_huggingface(readme_path, self.hf_repo_id, "README.md")
            print("✓ Model card uploaded")

        except Exception as e:
            print(f"Warning: Could not create model card: {e}")


print("✓ CheckpointAndUploadCallback defined")

✓ CheckpointAndUploadCallback defined


## Checkpoint Management & Hugging Face Integration

To handle Colab's time limits and enable resumable training, we'll implement:
1. **Full checkpoint saving**: Model weights + optimizer state + epoch number
2. **Hugging Face Hub integration**: Upload/download checkpoints
3. **Resume training**: Load checkpoints and continue from where we left off

### Code Block 11: Train Model


In [57]:
# ============================================================================
# TRAIN MODEL WITH CHECKPOINT SUPPORT
# ============================================================================

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_path = os.path.join(RESULTS_DIR, f"model_{timestamp}.keras")

# Setup callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=EARLY_STOPPING_PATIENCE,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=REDUCE_LR_FACTOR,
        patience=REDUCE_LR_PATIENCE,
        min_lr=MIN_LEARNING_RATE
    ),
    keras.callbacks.ModelCheckpoint(
        model_path,
        monitor='val_binary_accuracy',
        save_best_only=True
    ),
    # Add our custom checkpoint callback
    CheckpointAndUploadCallback(
        checkpoint_dir=CHECKPOINT_DIR,
        save_freq=CHECKPOINT_FREQ,
        upload_to_hf=UPLOAD_TO_HF,
        hf_repo_id=HF_MODEL_REPO if UPLOAD_TO_HF else None,
        keep_last=CHECKPOINT_KEEP_LAST  # Keep only last 3 checkpoints to save space
    )
]

print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80)
print(f"Training configuration:")
print(f"  Initial epoch: {initial_epoch}")
print(f"  Total epochs: {TOTAL_EPOCHS}")
print(f"  Epochs to train: {TOTAL_EPOCHS - initial_epoch}")
print(f"  Checkpoint every: {CHECKPOINT_FREQ} epochs")
print(f"  Upload to HF: {UPLOAD_TO_HF}")
if UPLOAD_TO_HF:
    print(f"  HF repo: {HF_MODEL_REPO}")
print("="*80 + "\n")

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=TOTAL_EPOCHS,
    initial_epoch=initial_epoch,  # Start from loaded epoch if resuming
    callbacks=callbacks,
    verbose=1
)

print(f"\n✓ Training complete!")
print(f"✓ Best model saved: {model_path}")
print(f"✓ Checkpoints saved in: {CHECKPOINT_DIR}")


STARTING TRAINING
Training configuration:
  Initial epoch: 21
  Total epochs: 50
  Epochs to train: 29
  Checkpoint every: 3 epochs
  Upload to HF: True
  HF repo: Neural-Network-Project/ECG-models

Epoch 22/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 150s 208ms/step - binary_accuracy: 0.9707 - loss: 0.0866 - val_binary_accuracy: 0.9642 - val_loss: 0.1085 - learning_rate: 0.0010
Epoch 23/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 112s 187ms/step - binary_accuracy: 0.9722 - loss: 0.0793 - val_binary_accuracy: 0.9596 - val_loss: 0.1127 - learning_rate: 0.0010
Epoch 24/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - binary_accuracy: 0.9728 - loss: 0.0791Saving checkpoint to ./checkpoints/checkpoint_epoch_024.pkl...
  Note: Optimizer weights not available yet (first epoch not complete)
✓ Checkpoint saved: ./checkpoints/checkpoint_epoch_024.pkl
✓ Model saved: ./checkpoints/checkpoint_epoch_024.keras
Uploading checkpoint to Hugging Face...
Uploading ./checkpoints/checkpoint_epoch_024.pkl to Neural-Network-Project

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  .../checkpoint_epoch_024.pkl:  28%|##8       |  554kB / 1.97MB            

Uploaded to: https://huggingface.co/Neural-Network-Project/ECG-models
  File: checkpoint_epoch_024.pkl
Uploading ./checkpoints/checkpoint_epoch_024.keras to Neural-Network-Project/ECG-models/checkpoint_epoch_024.keras...


...

Uploaded to: https://huggingface.co/Neural-Network-Project/ECG-models
  File: checkpoint_final.pkl
Uploading ./checkpoints/checkpoint_final.keras to Neural-Network-Project/ECG-models/checkpoint_final.keras...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ts/checkpoint_final.keras:  32%|###2      | 1.92MB / 5.99MB            

Uploaded to: https://huggingface.co/Neural-Network-Project/ECG-models
  File: checkpoint_final.keras
Uploading ./checkpoints/README.md to Neural-Network-Project/ECG-models/README.md...
Uploaded to: https://huggingface.co/Neural-Network-Project/ECG-models
  File: README.md
✓ Model card uploaded

✓ Training complete!
✓ Best model saved: ./results/model_20251124_214857.keras
✓ Checkpoints saved in: ./checkpoints/


In [61]:

api = HfApi()

try:
    api.upload_folder(
        folder_path=RESULTS_DIR,  # "./results/"
        repo_id="Neural-Network-Project/ECG-models",
        repo_type="model",
        path_in_repo="training_results",
        commit_message=f"Training results - {datetime.now().strftime('%Y%m%d_%H%M%S')}"
    )
    print("✓ Results successfully uploaded to HuggingFace!")
    print("View at: https://huggingface.co/Neural-Network-Project/ECG-models")
except Exception as e:
    print(f"❌ Upload failed: {str(e)}")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...del_20251124_214857.keras: 100%|##########| 5.99MB / 5.99MB            

  ...del_20251124_211549.keras:   5%|4         |  278kB / 5.99MB            

  ...del_20251124_213345.keras:   5%|4         |  278kB / 5.99MB            

✓ Results successfully uploaded to HuggingFace!
View at: https://huggingface.co/Neural-Network-Project/ECG-models


---

## 📈 Evaluation & Analysis

Comprehensive evaluation of the trained model on the test set.


### Code Block 13: Evaluate on Test Set


In [62]:
# Evaluate on test dataset with comprehensive metrics
print("Evaluating on test set...")
y_test_list = []
y_pred_list = []
y_probs_list = []

for X_batch, y_batch in test_dataset:
    y_test_list.append(y_batch.numpy())
    y_probs = model.predict(X_batch, verbose=0)
    y_probs_list.append(y_probs)
    y_pred = (y_probs >= DEFAULT_THRESHOLD).astype(int)
    y_pred_list.append(y_pred)

# Concatenate all batches
y_test = np.concatenate(y_test_list, axis=0)
y_pred = np.concatenate(y_pred_list, axis=0)
y_pred_probs = np.concatenate(y_probs_list, axis=0)

print(f"\n✓ Test set size: {len(y_test)} samples")

# ============================================================================
# COMPREHENSIVE EVALUATION METRICS
# ============================================================================

print("\n" + "="*80)
print("MULTI-LABEL CLASSIFICATION METRICS")
print("="*80)

# Global Metrics
hamming = hamming_loss(y_test, y_pred)
exact_match = np.mean(np.all(y_test == y_pred, axis=1))
subset_accuracy = jaccard_score(y_test, y_pred, average='samples', zero_division=0)
micro_f1 = f1_score(y_test, y_pred, average='micro', zero_division=0)
macro_f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

print(f"\n1. GLOBAL METRICS:")
print(f"   Hamming Loss (lower is better):      {hamming:.4f}")
print(f"   Exact Match Accuracy (all correct):  {exact_match:.4f}")
print(f"   Subset Accuracy (Jaccard Index):     {subset_accuracy:.4f}")
print(f"   Micro-averaged F1:                   {micro_f1:.4f}")
print(f"   Macro-averaged F1:                   {macro_f1:.4f}")

# Per-class metrics
print(f"\n2. PER-CLASS METRICS:")
print(f"{'Class':<20} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'ROC-AUC':>10} {'Support':>10}")
print("-" * 80)

per_class_results = {}
roc_auc_scores = []

for i, cls in enumerate(DISEASE_CLASSES):
    p = precision_score(y_test[:, i], y_pred[:, i], zero_division=0)
    r = recall_score(y_test[:, i], y_pred[:, i], zero_division=0)
    f = f1_score(y_test[:, i], y_pred[:, i], zero_division=0)

    # ROC-AUC (needs probabilities, not binary predictions)
    try:
        roc_auc = roc_auc_score(y_test[:, i], y_pred_probs[:, i])
    except:
        roc_auc = 0.0  # If only one class present
    roc_auc_scores.append(roc_auc)

    support = int(y_test[:, i].sum())

    per_class_results[cls] = {
        'precision': p, 'recall': r, 'f1': f, 'roc_auc': roc_auc, 'support': support
    }

    print(f"{cls:<20} {p:>10.4f} {r:>10.4f} {f:>10.4f} {roc_auc:>10.4f} {support:>10}")

print(f"\nMean ROC-AUC across classes:             {np.mean(roc_auc_scores):.4f}")

# Confusion matrices per class
print(f"\n3. CONFUSION MATRICES (per class, as 2x2 binary classification):")
print()

cm_data = []
for i, cls in enumerate(DISEASE_CLASSES):
    cm = confusion_matrix(y_test[:, i], y_pred[:, i])
    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else [cm[0, 0], 0, 0, cm[1, 1]]

    print(f"{cls}:")
    print(f"  True Negatives:  {int(tn):>6} | False Positives: {int(fp):>6}")
    print(f"  False Negatives: {int(fn):>6} | True Positives:  {int(tp):>6}")
    print()

    cm_data.append({'class': cls, 'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})

# Class distribution
print(f"4. CLASS DISTRIBUTION IN TEST SET:")
print()
for i, cls in enumerate(DISEASE_CLASSES):
    count = int(y_test[:, i].sum())
    pct = 100 * count / len(y_test)
    print(f"   {cls:<20}: {count:>6} ({pct:>5.1f}%)")

print("\n" + "="*80)

Evaluating on test set...

✓ Test set size: 11989 samples

MULTI-LABEL CLASSIFICATION METRICS

1. GLOBAL METRICS:
   Hamming Loss (lower is better):      0.0245
   Exact Match Accuracy (all correct):  0.9368
   Subset Accuracy (Jaccard Index):     0.9372
   Micro-averaged F1:                   0.9386
   Macro-averaged F1:                   0.7480

2. PER-CLASS METRICS:
Class                 Precision     Recall   F1-Score    ROC-AUC    Support
--------------------------------------------------------------------------------
Myocarditis              0.8395     0.7094     0.7690     0.9729        413
Cardiomyopathy           0.9245     0.9074     0.9159     0.9983         54
Kawasaki                 0.6667     0.2353     0.3478     0.9647         68
CHD                      0.7612     0.7221     0.7411     0.9529       1011
Healthy                  0.9597     0.9722     0.9659     0.9508      10443

Mean ROC-AUC across classes:             0.9679

3. CONFUSION MATRICES (per class, as 2x2 

### Code Block 14: Display Comparison Tables


In [63]:
# Save optimized threshold results
threshold_results = {
    'timestamp': timestamp,
    'disease_classes': DISEASE_CLASSES,

    # Optimized thresholds for each strategy
    'optimized_thresholds': {
        'f1_max': optimal_thresholds['f1_max'],
        'youden': optimal_thresholds['youden'],
        'recall_priority': optimal_thresholds['recall_priority']
    },

    # Comparison results
    'comparison': {
        result['strategy']: {
            'global_metrics': result['global_metrics'],
            'per_class_metrics': {
                cls: {
                    'precision': float(result['per_class'][cls]['precision']),
                    'recall': float(result['per_class'][cls]['recall']),
                    'f1': float(result['per_class'][cls]['f1']),
                    'threshold': float(result['per_class'][cls]['threshold'])
                }
                for cls in DISEASE_CLASSES
            }
        }
        for result in all_results
    }
}

# Save results
threshold_results_path = os.path.join(RESULTS_DIR, f"threshold_optimization_{timestamp}.json")
with open(threshold_results_path, "w") as f:
    json.dump(threshold_results, f, indent=2)

print(f"✓ Threshold optimization results saved to: {threshold_results_path}")

# Save comparison as CSV for easy reference
comparison_data = []
for result in all_results:
    for cls in DISEASE_CLASSES:
        comparison_data.append({
            'Strategy': result['strategy'],
            'Class': cls,
            'Threshold': result['per_class'][cls]['threshold'],
            'Precision': result['per_class'][cls]['precision'],
            'Recall': result['per_class'][cls]['recall'],
            'F1-Score': result['per_class'][cls]['f1']
        })

comparison_df = pd.DataFrame(comparison_data)
comparison_csv_path = os.path.join(RESULTS_DIR, f"threshold_comparison_{timestamp}.csv")
comparison_df.to_csv(comparison_csv_path, index=False)

print(f"✓ Comparison table saved to: {comparison_csv_path}")

NameError: name 'optimal_thresholds' is not defined

### Code Block 15: Analysis Summary


In [ ]:
# Analysis summary and recommendations
print("\n" + "="*80)
print("THRESHOLD OPTIMIZATION SUMMARY & RECOMMENDATIONS")
print("="*80)

# Find best strategy for each class based on F1-score
print("\n1. BEST STRATEGY PER CLASS (by F1-Score):")
print("-" * 80)
for cls in DISEASE_CLASSES:
    best_strategy = None
    best_f1 = 0.0

    for result in all_results:
        f1 = result['per_class'][cls]['f1']
        if f1 > best_f1:
            best_f1 = f1
            best_strategy = result['strategy']

    baseline_f1 = baseline_results['per_class'][cls]['f1']
    improvement = best_f1 - baseline_f1

    print(f"{cls:<20}: {best_strategy:<25} (F1: {best_f1:.4f}, +{improvement:+.4f} vs baseline)")

# Calculate improvements for Kawasaki (the most problematic class)
print(f"\n2. KAWASAKI DISEASE IMPROVEMENTS:")
print("-" * 80)
baseline_kawasaki = baseline_results['per_class']['Kawasaki']
print(f"{'Strategy':<25} {'Recall':>10} {'Δ Recall':>10} {'F1':>10} {'Δ F1':>10}")
print("-" * 70)

for result in all_results:
    kawa = result['per_class']['Kawasaki']
    recall_delta = kawa['recall'] - baseline_kawasaki['recall']
    f1_delta = kawa['f1'] - baseline_kawasaki['f1']

    print(f"{result['strategy']:<25} {kawa['recall']:>10.4f} {recall_delta:>+10.4f} "
          f"{kawa['f1']:>10.4f} {f1_delta:>+10.4f}")

# Overall macro-F1 improvements
print(f"\n3. OVERALL PERFORMANCE (Macro-F1):")
print("-" * 80)
print(f"{'Strategy':<25} {'Macro-F1':>10} {'Δ vs Baseline':>15}")
print("-" * 70)

baseline_macro = baseline_results['global_metrics']['macro_f1']
for result in all_results:
    macro_f1 = result['global_metrics']['macro_f1']
    delta = macro_f1 - baseline_macro

    print(f"{result['strategy']:<25} {macro_f1:>10.4f} {delta:>+15.4f}")

print("\n" + "="*80)
print("RECOMMENDATION:")
print("="*80)

# Find best overall strategy
best_overall = max(all_results, key=lambda x: x['global_metrics']['macro_f1'])
print(f"\nBest overall strategy: {best_overall['strategy']}")
print(f"  Macro-F1: {best_overall['global_metrics']['macro_f1']:.4f}")
print(f"  Improvement: +{best_overall['global_metrics']['macro_f1'] - baseline_macro:.4f}")

print("\n✓ Threshold optimization analysis complete!")
print("\nNext steps:")
print("  - If Kawasaki recall is still too low, consider:")
print("    1. Class weighting or focal loss (Phase 2)")
print("    2. Data augmentation for minority classes")
print("    3. Oversampling techniques (SMOTE)")
print("  - Save the optimized thresholds for production use")
print("  - Consider ensembling multiple threshold strategies")

### Code Block 16: Visualize Threshold Optimization


In [ ]:
# Visualization 1: Per-class F1-Score comparison across strategies
print("Creating threshold optimization visualizations...")

fig, axes = plt.subplots(2, 2, figsize=FIGSIZE_LARGE)

# 1. F1-Score comparison (top-left)
strategies = [r['strategy'] for r in all_results]
x = np.arange(len(DISEASE_CLASSES))
width = 0.2

for i, result in enumerate(all_results):
    f1_scores = [result['per_class'][cls]['f1'] for cls in DISEASE_CLASSES]
    axes[0, 0].bar(x + i*width, f1_scores, width, label=result['strategy'], alpha=0.8)

axes[0, 0].set_xlabel('Disease Class')
axes[0, 0].set_ylabel('F1-Score')
axes[0, 0].set_title('F1-Score Comparison Across Threshold Strategies')
axes[0, 0].set_xticks(x + width * 1.5)
axes[0, 0].set_xticklabels(DISEASE_CLASSES, rotation=15, ha='right')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='y')
axes[0, 0].set_ylim([0, 1])

# 2. Recall comparison (top-right)
for i, result in enumerate(all_results):
    recall_scores = [result['per_class'][cls]['recall'] for cls in DISEASE_CLASSES]
    axes[0, 1].bar(x + i*width, recall_scores, width, label=result['strategy'], alpha=0.8)

axes[0, 1].set_xlabel('Disease Class')
axes[0, 1].set_ylabel('Recall')
axes[0, 1].set_title('Recall Comparison Across Threshold Strategies')
axes[0, 1].set_xticks(x + width * 1.5)
axes[0, 1].set_xticklabels(DISEASE_CLASSES, rotation=15, ha='right')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis='y')
axes[0, 1].set_ylim([0, 1])

# 3. Precision comparison (bottom-left)
for i, result in enumerate(all_results):
    precision_scores = [result['per_class'][cls]['precision'] for cls in DISEASE_CLASSES]
    axes[1, 0].bar(x + i*width, precision_scores, width, label=result['strategy'], alpha=0.8)

axes[1, 0].set_xlabel('Disease Class')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].set_title('Precision Comparison Across Threshold Strategies')
axes[1, 0].set_xticks(x + width * 1.5)
axes[1, 0].set_xticklabels(DISEASE_CLASSES, rotation=15, ha='right')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')
axes[1, 0].set_ylim([0, 1])

# 4. Optimized thresholds (bottom-right)
for i, (strategy_name, thresholds) in enumerate([
    ('F1-Max', optimal_thresholds['f1_max']),
    ('Youden', optimal_thresholds['youden']),
    ('Recall-Priority', optimal_thresholds['recall_priority'])
]):
    threshold_values = [thresholds[cls] for cls in DISEASE_CLASSES]
    axes[1, 1].bar(x + i*width, threshold_values, width, label=strategy_name, alpha=0.8)

# Add baseline threshold line
axes[1, 1].axhline(y=0.5, color='red', linestyle='--', linewidth=2, label='Baseline (0.5)', alpha=0.7)

axes[1, 1].set_xlabel('Disease Class')
axes[1, 1].set_ylabel('Threshold')
axes[1, 1].set_title('Optimized Thresholds by Strategy')
axes[1, 1].set_xticks(x + width * 1.5)
axes[1, 1].set_xticklabels(DISEASE_CLASSES, rotation=15, ha='right')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')
axes[1, 1].set_ylim([0, 1])

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, f'threshold_optimization_{timestamp}.png'), dpi=DPI, bbox_inches='tight')
print(f"✓ Threshold optimization comparison saved")
plt.show()

### Code Block 17: Display Comparison Tables


In [ ]:
# Display comparison tables
print("\n" + "="*80)
print("COMPARISON: GLOBAL METRICS")
print("="*80)
print(f"\n{'Strategy':<25} {'Hamming':>10} {'Exact':>10} {'Jaccard':>10} {'Micro-F1':>10} {'Macro-F1':>10}")
print("-" * 80)

for result in all_results:
    gm = result['global_metrics']
    print(f"{result['strategy']:<25} {gm['hamming_loss']:>10.4f} {gm['exact_match']:>10.4f} "
          f"{gm['subset_accuracy']:>10.4f} {gm['micro_f1']:>10.4f} {gm['macro_f1']:>10.4f}")

print("\n" + "="*80)
print("COMPARISON: PER-CLASS METRICS")
print("="*80)

for cls in DISEASE_CLASSES:
    print(f"\n{cls}:")
    print(f"{'Strategy':<25} {'Threshold':>10} {'Precision':>10} {'Recall':>10} {'F1-Score':>10}")
    print("-" * 70)

    for result in all_results:
        pc = result['per_class'][cls]
        print(f"{result['strategy']:<25} {pc['threshold']:>10.3f} {pc['precision']:>10.4f} "
              f"{pc['recall']:>10.4f} {pc['f1']:>10.4f}")

print("\n" + "="*80)

### Code Block 18: Evaluate with Optimized Thresholds


In [ ]:
# Evaluate test set with optimized thresholds
def evaluate_with_thresholds(y_true, y_probs, thresholds_dict, strategy_name):
    """Evaluate test set using per-class optimized thresholds."""

    # Apply per-class thresholds
    y_pred_optimized = np.zeros_like(y_probs)
    for i, cls in enumerate(DISEASE_CLASSES):
        threshold = thresholds_dict[cls]
        y_pred_optimized[:, i] = (y_probs[:, i] >= threshold).astype(int)

    y_pred_optimized = y_pred_optimized.astype(int)

    # Calculate global metrics
    hamming = hamming_loss(y_true, y_pred_optimized)
    exact_match = np.mean(np.all(y_true == y_pred_optimized, axis=1))
    subset_accuracy = jaccard_score(y_true, y_pred_optimized, average='samples', zero_division=0)
    micro_f1 = f1_score(y_true, y_pred_optimized, average='micro', zero_division=0)
    macro_f1 = f1_score(y_true, y_pred_optimized, average='macro', zero_division=0)

    # Calculate per-class metrics
    per_class = {}
    for i, cls in enumerate(DISEASE_CLASSES):
        p = precision_score(y_true[:, i], y_pred_optimized[:, i], zero_division=0)
        r = recall_score(y_true[:, i], y_pred_optimized[:, i], zero_division=0)
        f = f1_score(y_true[:, i], y_pred_optimized[:, i], zero_division=0)

        per_class[cls] = {
            'precision': p,
            'recall': r,
            'f1': f,
            'threshold': thresholds_dict[cls]
        }

    return {
        'strategy': strategy_name,
        'global_metrics': {
            'hamming_loss': hamming,
            'exact_match': exact_match,
            'subset_accuracy': subset_accuracy,
            'micro_f1': micro_f1,
            'macro_f1': macro_f1
        },
        'per_class': per_class,
        'predictions': y_pred_optimized
    }


print("\n" + "="*80)
print("EVALUATING TEST SET WITH OPTIMIZED THRESHOLDS")
print("="*80)

# Baseline (0.5 threshold for all classes)
baseline_thresholds = {cls: 0.5 for cls in DISEASE_CLASSES}
baseline_results = evaluate_with_thresholds(y_test, y_pred_probs, baseline_thresholds, "Baseline (0.5)")

# Optimized strategies
f1_max_results = evaluate_with_thresholds(y_test, y_pred_probs, optimal_thresholds['f1_max'], "F1-Maximization")
youden_results = evaluate_with_thresholds(y_test, y_pred_probs, optimal_thresholds['youden'], "Youden's J")
recall_priority_results = evaluate_with_thresholds(y_test, y_pred_probs, optimal_thresholds['recall_priority'], "Recall-Prioritized")

all_results = [baseline_results, f1_max_results, youden_results, recall_priority_results]

print("\n✓ Test set evaluated with all threshold strategies")

---

## 🎚️ Threshold Optimization

Optimize classification thresholds for each disease class to improve performance.


### Code Block 19: Get Validation Predictions


In [ ]:
# Find optimal thresholds for each class using validation set
print("Finding optimal thresholds for each class...")
print("="*80)

optimal_thresholds = {
    'f1_max': {},
    'youden': {},
    'recall_priority': {}
}

# Different min_recall targets for different disease severities
# Kawasaki and Cardiomyopathy are most critical (rare + serious)
# Myocarditis and CHD are also important but more common
# Min recall targets defined in config
min_recall_targets = MIN_RECALL_TARGETS

for i, cls in enumerate(DISEASE_CLASSES):
    print(f"\n{cls}:")
    print(f"  Support in validation set: {int(y_val[:, i].sum())}")

    # Strategy 1: Maximize F1-score
    thresh_f1, score_f1 = optimize_threshold_f1(y_val[:, i], y_val_probs[:, i])
    optimal_thresholds['f1_max'][cls] = thresh_f1
    print(f"  F1-max strategy:        threshold = {thresh_f1:.3f}, F1 = {score_f1:.4f}")

    # Strategy 2: Youden's J statistic
    thresh_youden, score_youden = optimize_threshold_youden(y_val[:, i], y_val_probs[:, i])
    optimal_thresholds['youden'][cls] = thresh_youden
    print(f"  Youden's J strategy:    threshold = {thresh_youden:.3f}, J = {score_youden:.4f}")

    # Strategy 3: Recall-prioritized (different targets per class)
    min_recall = min_recall_targets[cls]
    thresh_recall, score_recall = optimize_threshold_recall_prioritized(
        y_val[:, i], y_val_probs[:, i], min_recall=min_recall
    )
    optimal_thresholds['recall_priority'][cls] = thresh_recall

    # Check actual recall achieved
    y_pred_recall = (y_val_probs[:, i] >= thresh_recall).astype(int)
    actual_recall = recall_score(y_val[:, i], y_pred_recall, zero_division=0)
    actual_f1 = f1_score(y_val[:, i], y_pred_recall, zero_division=0)
    print(f"  Recall-priority (≥{min_recall:.0%}): threshold = {thresh_recall:.3f}, recall = {actual_recall:.4f}, F1 = {actual_f1:.4f}")

print("\n" + "="*80)
print("\n✓ Optimal thresholds found for all classes")

### Code Block 20: Threshold Optimization Functions


In [ ]:
# Threshold optimization functions
def optimize_threshold_f1(y_true, y_probs, thresholds=np.arange(THRESHOLD_SEARCH_START, THRESHOLD_SEARCH_END, THRESHOLD_SEARCH_STEP)):
    """Find threshold that maximizes F1-score."""
    best_threshold = 0.5
    best_f1 = 0.0

    for threshold in thresholds:
        y_pred = (y_probs >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1


def optimize_threshold_youden(y_true, y_probs, thresholds=np.arange(THRESHOLD_SEARCH_START, THRESHOLD_SEARCH_END, THRESHOLD_SEARCH_STEP)):
    """Find threshold that maximizes Youden's J statistic (sensitivity + specificity - 1)."""
    best_threshold = 0.5
    best_j = -1.0

    for threshold in thresholds:
        y_pred = (y_probs >= threshold).astype(int)

        # Calculate confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        if cm.size == 4:
            tn, fp, fn, tp = cm.ravel()
        else:
            continue

        # Calculate sensitivity (recall) and specificity
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

        # Youden's J statistic
        j_stat = sensitivity + specificity - 1

        if j_stat > best_j:
            best_j = j_stat
            best_threshold = threshold

    return best_threshold, best_j


def optimize_threshold_recall_prioritized(y_true, y_probs, min_recall=0.85, thresholds=np.arange(THRESHOLD_SEARCH_START-0.05, THRESHOLD_SEARCH_END, THRESHOLD_SEARCH_STEP)):
    """
    Find threshold that maximizes F1-score while maintaining minimum recall.
    This is useful for medical diagnosis where missing cases is costly.
    """
    best_threshold = 0.5
    best_f1 = 0.0

    for threshold in thresholds:
        y_pred = (y_probs >= threshold).astype(int)

        recall = recall_score(y_true, y_pred, zero_division=0)

        # Only consider thresholds that meet minimum recall requirement
        if recall >= min_recall:
            f1 = f1_score(y_true, y_pred, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold

    # If no threshold meets min_recall, just maximize recall
    if best_f1 == 0.0:
        best_recall = 0.0
        for threshold in thresholds:
            y_pred = (y_probs >= threshold).astype(int)
            recall = recall_score(y_true, y_pred, zero_division=0)
            if recall > best_recall:
                best_recall = recall
                best_threshold = threshold
                best_f1 = f1_score(y_true, y_pred, zero_division=0)

    return best_threshold, best_f1


print("✓ Threshold optimization functions defined")

### Code Block 21: Get Validation Predictions


In [ ]:
# Get predictions on validation set for threshold optimization
print("Getting validation set predictions for threshold optimization...")
y_val_list = []
y_val_probs_list = []

for X_batch, y_batch in val_dataset:
    y_val_list.append(y_batch.numpy())
    y_probs = model.predict(X_batch, verbose=0)
    y_val_probs_list.append(y_probs)

y_val = np.concatenate(y_val_list, axis=0)
y_val_probs = np.concatenate(y_val_probs_list, axis=0)

print(f"✓ Validation set: {len(y_val)} samples")
print(f"  Probabilities shape: {y_val_probs.shape}")

## Threshold Optimization

The baseline evaluation above uses a fixed threshold of 0.5 for all classes. However, given the severe class imbalance (87% healthy vs 0.5-8.4% diseases), optimizing per-class thresholds can significantly improve performance, especially for minority classes like Kawasaki disease.

We'll optimize thresholds using the validation set and then re-evaluate on the test set.

---

## 📊 Visualizations

Generate comprehensive visualizations of model performance.


### Code Block 22: Generate ROC Curves


In [ ]:
# Evaluation Visualizations
print("Creating evaluation visualizations...")

# 1. ROC Curves per class
fig, axes = plt.subplots(2, 3, figsize=FIGSIZE_ROC)
axes = axes.flatten()

all_fpr = []
all_tpr = []
all_roc_auc = []

for i, cls in enumerate(DISEASE_CLASSES):
    fpr, tpr, _ = roc_curve(y_test[:, i], y_pred_probs[:, i])
    roc_auc = auc(fpr, tpr)

    all_fpr.append(fpr)
    all_tpr.append(tpr)
    all_roc_auc.append(roc_auc)

    axes[i].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
    axes[i].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance')
    axes[i].set_xlim([0.0, 1.0])
    axes[i].set_ylim([0.0, 1.05])
    axes[i].set_xlabel('False Positive Rate')
    axes[i].set_ylabel('True Positive Rate')
    axes[i].set_title(f'{cls}')
    axes[i].legend(loc="lower right")
    axes[i].grid(True, alpha=0.3)

# Remove extra subplot
axes[-1].axis('off')

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, f'roc_curves_{timestamp}.png'), dpi=DPI, bbox_inches='tight')
print(f"✓ ROC curves saved")
plt.show()

# 2. Confusion Matrices per class
fig, axes = plt.subplots(2, 3, figsize=FIGSIZE_ROC)
axes = axes.flatten()

for i, cls in enumerate(DISEASE_CLASSES):
    cm = confusion_matrix(y_test[:, i], y_pred[:, i])
    if cm.size == 4:
        tn, fp, fn, tp = cm.ravel()
    else:
        tn, fp, fn, tp = cm[0, 0], 0, 0, cm[1, 1] if cm.shape[0] > 1 else 0

    # Normalize for better visualization
    cm_display = np.array([[tn, fp], [fn, tp]])

    sns.heatmap(cm_display, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                cbar=False, xticklabels=['Negative', 'Positive'],
                yticklabels=['Negative', 'Positive'])
    axes[i].set_title(f'{cls}')
    axes[i].set_ylabel('True Label')
    axes[i].set_xlabel('Predicted Label')

# Remove extra subplot
axes[-1].axis('off')

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, f'confusion_matrices_{timestamp}.png'), dpi=DPI, bbox_inches='tight')
print(f"✓ Confusion matrices saved")
plt.show()

# 3. Metrics summary bar chart
fig, axes = plt.subplots(1, 2, figsize=FIGSIZE_MEDIUM)

# Per-class F1 scores
f1_scores = [per_class_results[cls]['f1'] for cls in DISEASE_CLASSES]
axes[0].bar(DISEASE_CLASSES, f1_scores, color='steelblue', alpha=0.8)
axes[0].set_ylabel('F1-Score')
axes[0].set_title('Per-Class F1-Score')
axes[0].set_ylim([0, 1])
axes[0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(f1_scores):
    axes[0].text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom')

# Per-class ROC-AUC
roc_aucs = [per_class_results[cls]['roc_auc'] for cls in DISEASE_CLASSES]
axes[1].bar(DISEASE_CLASSES, roc_aucs, color='coral', alpha=0.8)
axes[1].set_ylabel('ROC-AUC')
axes[1].set_title('Per-Class ROC-AUC Score')
axes[1].set_ylim([0, 1])
axes[1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(roc_aucs):
    axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, f'metrics_summary_{timestamp}.png'), dpi=DPI, bbox_inches='tight')
print(f"✓ Metrics summary saved")
plt.show()

print("✓ All evaluation visualizations created")

### Code Block 24: Save Predictions & Results


In [ ]:
# Save predictions and comprehensive results
np.savez(
    os.path.join(RESULTS_DIR, f"predictions_{timestamp}.npz"),
    y_true=y_test,
    y_pred=y_pred,
    y_probs=y_pred_probs
)

# Save comprehensive results JSON
results = {
    'timestamp': timestamp,
    'disease_classes': DISEASE_CLASSES,
    'epochs_trained': len(history.history['loss']),
    'test_samples': len(y_test),

    # Global metrics
    'global_metrics': {
        'hamming_loss': float(hamming),
        'exact_match_accuracy': float(exact_match),
        'subset_accuracy': float(subset_accuracy),
        'micro_f1': float(micro_f1),
        'macro_f1': float(macro_f1),
        'mean_roc_auc': float(np.mean(roc_auc_scores))
    },

    # Per-class metrics
    'per_class_metrics': {
        cls: {
            'precision': float(per_class_results[cls]['precision']),
            'recall': float(per_class_results[cls]['recall']),
            'f1_score': float(per_class_results[cls]['f1']),
            'roc_auc': float(per_class_results[cls]['roc_auc']),
            'support': int(per_class_results[cls]['support'])
        }
        for cls in DISEASE_CLASSES
    }
}

# Save results
with open(os.path.join(RESULTS_DIR, f"results_{timestamp}.json"), "w") as f:
    json.dump(results, f, indent=2)

# Save results summary as CSV
results_df = pd.DataFrame([
    {
        'Class': cls,
        'Precision': per_class_results[cls]['precision'],
        'Recall': per_class_results[cls]['recall'],
        'F1-Score': per_class_results[cls]['f1'],
        'ROC-AUC': per_class_results[cls]['roc_auc'],
        'Support': per_class_results[cls]['support']
    }
    for cls in DISEASE_CLASSES
])

results_df.to_csv(os.path.join(RESULTS_DIR, f"evaluation_metrics_{timestamp}.csv"), index=False)

print(f"\n✓ Results saved to {RESULTS_DIR}")
print(f"  - predictions_{timestamp}.npz (y_true, y_pred, y_probs)")
print(f"  - results_{timestamp}.json (comprehensive metrics)")
print(f"  - evaluation_metrics_{timestamp}.csv (per-class metrics table)")
print(f"  - model_{timestamp}.keras (trained model)")
print(f"  - roc_curves_{timestamp}.png")
print(f"  - confusion_matrices_{timestamp}.png")
print(f"  - metrics_summary_{timestamp}.png")